In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [23]:
pd.set_option('max_rows', 20)

In [2]:
data_file = 'source/GDL-AreaData200.csv'
desc_file = 'source/GDL-AreaData200-Variabledescription.csv'

In [299]:
data = pd.read_csv(data_file, encoding='iso-8859-1', skipinitialspace=True)

In [59]:
data

,iso_code,ISO2,country,year,datasource,GDLCODE,level,region,iwi,iwipov50,...,wastingsev,whz,overweight,obesity,bmiz,popshare,totpopm,regpopm,N,Nhh
0,AFG,AF,Afghanistan,2010,MICS,AFGr101,Subnat,Central (Kabul Wardak Kapisa Logar Parwan Panj...,51.9,51.9,...,,,,,,16,27.962,4.474,16232,2159
1,AFG,AF,Afghanistan,2010,MICS,AFGr102,Subnat,Central Highlands (Bamyan Daikundi),37.8,83.1,...,,,,,,3.4,27.962,.951,3449,432
2,AFG,AF,Afghanistan,2010,MICS,AFGr103,Subnat,East (Nangarhar Kunar Laghman Nooristan),38.6,78.8,...,,,,,,11.1,27.962,3.104,11335,1520
3,AFG,AF,Afghanistan,2010,MICS,AFGr104,Subnat,North (Samangan Sar-e-Pul Balkh Jawzjan Faryab),39.6,77.8,...,,,,,,13.8,27.962,3.859,14055,1913
4,AFG,AF,Afghanistan,2010,MICS,AFGr105,Subnat,North East (Baghlan Takhar Badakhshan Kunduz),42.8,71,...,,,,,,16.3,27.962,4.558,16557,2091
5,AFG,AF,Afghanistan,2010,MICS,AFGr106,Subnat,South (Uruzgan Helmand Zabul Nimroz Kandahar),41.5,73.5,...,,,,,,13.6,27.962,3.803,13825,1584
6,AFG,AF,Afghanistan,2010,MICS,AFGr107,Subnat,South East (Ghazni Paktya Paktika Khost),47.6,52.5,...,,,,,,12.7,27.962,3.551,12867,1263
7,AFG,AF,Afghanistan,2010,MICS,AFGr108,Subnat,West (Ghor Herat Badghis Farah),43,68.3,...,,,,,,13.2,27.962,3.691,13393,2155
8,AFG,AF,Afghanistan,2010,MICS,AFGt,National,Total,43.5,68.2,...,,,,,,100,27.962,27.962,101713,13116
9,AFG,AF,Afghanistan,2010,MICS,AFGtr,Urb/rur,Rural,38.8,78.2,...,,,,,,82.3,27.962,23.013,83713,10689


In [ ]:
# region

In [143]:
region = data[['GDLCODE', 'iso_code', 'ISO2', 'country', 'region', 'level']].drop_duplicates().copy()

In [144]:
region['region_name'] = region['country'] + ' - ' + region['region']

In [145]:
region['GDLCODE'] = region['GDLCODE'].apply(to_concept_id)

In [146]:
np.all(region['GDLCODE'].duplicated())

False

In [308]:
region['is--country'] = 0
region['is--subregion'] = 0
region['is--urb_rur'] = 0

In [309]:
region.loc[region['level'] == 'National', 'is--country'] = 1
region.loc[region['level'] == 'Subnat', 'is--sub_nation'] = 1
region.loc[region['level'] == 'Urb/rur', 'is--urb_rur'] = 1

In [347]:
region_ddf = region[['GDLCODE', 'region_name', 'iso_code', 'ISO2', 'is--country', 'is--sub_nation', 'is--urb_rur']].copy()

In [348]:
region_ddf = region_ddf.rename(columns={'GDLCODE': 'region', 'region_name': 'name', 'ISO2': 'iso2'})

In [50]:
# region_ddf.loc[region_ddf['is--country'] != 'TRUE', ['iso_code', 'iso2']] = np.nan

In [349]:
region_ddf.to_csv('../ddf--entities--region.csv', index=False)

In [55]:
# concepts

In [328]:
desc = pd.read_csv(desc_file, encoding='iso-8859-1')

In [329]:
desc['Variable'] = desc['Variable'].map(to_concept_id)

In [330]:
desc = desc.drop(['Category'], axis=1)

In [331]:
desc = desc.set_index('Variable')

desc = desc.drop(['datasource', 'level'])

In [332]:
desc = desc.ix[:'cookwood']

In [333]:
desc = desc.rename(index={'region': 'sub_nation', 'gdlcode': 'region'})

In [334]:
desc.columns = ['name', 'longdescr']

In [335]:
desc['concept_type'] = 'measure'

In [336]:
desc.loc[['iso_code', 'iso2'], 'concept_type'] = 'string'
desc.loc[['country', 'sub_nation'], 'concept_type'] = 'entity_set'
desc.loc['region', 'concept_type'] = 'entity_domain'
desc.loc['year', 'concept_type'] = 'time'

In [337]:
desc = desc.append(pd.DataFrame([['Urban/Rural Regions', '', 'entity_set'],
                                 ['Name', '', 'string']
                                ], columns=desc.columns, index=['urb_rur', 'name']))

In [338]:
desc['drillups'] = np.nan

desc.loc[['sub_nation', 'urb_rur'], 'drillups'] = 'country'
desc.loc['country', 'drillups'] = 'region'

In [339]:
desc['domain'] = np.nan

desc.loc[['sub_nation', 'urb_rur', 'country'], 'domain'] = 'region'

In [340]:
desc.index.name = 'concept'
desc = desc.reset_index()

In [273]:
'cookwood' in desc['concept'].values

True

In [274]:
desc.sort_values(by='concept_type')

,concept,name,longdescr,concept_type,drill_ups
4,region,Region code Global Data Lab,"Iso-code, level (t=total, u=urban, r=rural), r...",entity_domain,NaN
89,urb_rur,Urban/Rural Regions,,entity_set,country
2,country,Country name,NaN,entity_set,region
5,sub_nation,Sub-national region name,Name of sub-national region,entity_set,country
63,age4049,% population aged 40-49 in region,Percentage of national population aged 40-49 i...,measure,NaN
62,age3039,% population aged 30-39 in region,Percentage of national population aged 30-39 i...,measure,NaN
61,age2029,% population aged 20-29 in region,Percentage of national population aged 20-29 i...,measure,NaN
60,age1019,% population aged 10-19 in region,Percentage of national population aged 10-19 i...,measure,NaN
59,age09,% population aged 0-9 in region,Percentage of national population aged 0-9 in ...,measure,NaN
58,popshare,Share of population living in area,Share of national population living in area,measure,NaN


In [221]:
desc.sort_values(by='concept_type').to_csv('../ddf--concepts.csv', index=False)

In [ ]:
# datapoints

In [300]:
dps = data.copy()

In [301]:
dps.columns = list(map(to_concept_id, dps.columns))

In [302]:
dps = dps.rename(columns={ 'region': 'sub_nation', 'gdlcode': 'region',})

In [255]:
measures = desc[desc['concept_type'] == 'measure']['concept'].values

In [257]:
len(dps.columns[8:])

79

In [258]:
len(measures)

84

In [259]:
set(dps.columns[8:]).difference(set(measures))

{'totpopm'}

In [260]:
set(measures).difference(set(dps.columns[8:]))

{'chmort', 'infmort', 'niwi', 'nnmort', 'pnnmort', 'u5mort'}

In [341]:
desc = desc.set_index('concept')

In [342]:
idx1 = desc.ix[desc['concept_type'] != 'measure'].index
idx2 = dps.columns[8:]

In [343]:
idx = np.r_[idx1, idx2]

desc.ix[idx]

,name,longdescr,concept_type,drillups,domain
concept,,,,,
iso_code,ISO country code 3 digit,NaN,string,NaN,NaN
iso2,ISO regio code two digit,NaN,string,NaN,NaN
country,Country name,NaN,entity_set,region,region
year,Year of data collection,NaN,time,NaN,NaN
region,Region code Global Data Lab,"Iso-code, level (t=total, u=urban, r=rural), r...",entity_domain,NaN,NaN
sub_nation,Sub-national region name,Name of sub-national region,entity_set,country,region
urb_rur,Urban/Rural Regions,,entity_set,country,region
name,Name,,string,NaN,NaN
modsizehouse,% households with two sleeping rooms in region,Percentage of households with two sleeping roo...,measure,NaN,NaN


In [344]:
desc.loc['totpopm', 'concept_type'] = 'measure'

In [345]:
desc.sort_values(by='concept_type').to_csv('../ddf--concepts.csv')

In [303]:
dps = dps[['year', 'region', *dps.columns[8:]]]

In [304]:
dps['region'] = dps['region'].map(to_concept_id)

In [305]:
dps = dps.set_index(['year', 'region'])

In [306]:
for k, df in dps.items():
    df.columns = [k]
    df = df.dropna()
    df = df.reset_index()
    path = '../ddf--datapoints--{}--by--region--year.csv'.format(k)
    df.to_csv(path, index=False)

In [346]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,longdescr,ddf--concepts.csv
2,concept,concept_type,ddf--concepts.csv
3,concept,drillups,ddf--concepts.csv
4,concept,domain,ddf--concepts.csv
0,"region,year",age09,ddf--datapoints--age09--by--region--year.csv
0,"region,year",age1019,ddf--datapoints--age1019--by--region--year.csv
0,"region,year",age1chw20,ddf--datapoints--age1chw20--by--region--year.csv
0,"region,year",age2029,ddf--datapoints--age2029--by--region--year.csv
0,"region,year",age3039,ddf--datapoints--age3039--by--region--year.csv
